In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Codey API 入門 - Code Chat

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/code/code_chat.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br>在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/code/code_chat.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br>在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/code/code_chat.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br>在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Lavi Nigam](https://github.com/lavinigam-gcp), [Polong Lin](https://github.com/polong-lin) |


## 概觀

此筆記本提供了使用 [Codey chat 模型](https://cloud.google.com/vertex-ai/docs/generative-ai/code/code-models-overview)，特別是 codechat-bison 基礎模型進行程式碼聊天生成簡介。程式碼聊天讓開發人員能夠與聊天機器人互動，以取得程式碼相關工作支援，包括偵錯、撰寫文件和學習新概念。codechat-bison 模型旨在協助多輪對話，打造更自然互動的編碼體驗。


### Vertex AI PaLM API
在 [2023 年 5 月 10 日發布](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023) 的 Vertex AI PaLM API 支援 [PaLM 2](https://ai.google/discover/palm2)。

### 使用 Vertex AI PaLM API

你可以透過以下方式與 Vertex AI PaLM API 互動：

* 使用 [Generative AI Studio](https://cloud.google.com/generative-ai-studio) 進行快速測試和指令生成。
* 在 Cloud Shell 中使用 cURL 指令。
* 在 Jupyter 筆記本中使用 Python SDK

此筆記本著重於使用 Python SDK 呼叫 Vertex AI PaLM API。如需更多關於無需撰寫程式碼使用 Generative AI Studio 的資訊，你可以探索 [使用 UI 指導的入門事項](https://github.com/doggy8088/generative-ai/blob/main/language/intro_vertex_ai_studio.md)


如需更多資訊，請查閱 [Vertex AI Generative AI 支援文件](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)。


### 目標

在此教學中，你將會學習到：

* 程式除錯
* 程式重構
* 程式檢閱
* 程式學習
* 程式樣板
* 聊天問題提示設計
  * 驗證鏈
  * 自我一致性
  * 思想樹


### 費用
本教學指南使用 Google
Cloud 的計費元件：

* Vertex AI Generative AI Studio

了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)
，並使用 [定價計算器](https://cloud.google.com/products/calculator/)
，根據預計用量產生成本估算。


## 開始使用


### 安裝 Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --user

**僅限 Colab：** 取消以下Cell註解以重新啟動核心或使用按鈕重新啟動核心。對於 Vertex AI Workbench，你可以使用右上方的按鈕重新啟動終端機。


In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### 驗證筆記本環境
* 如果你使用 **Colab** 執行此筆記本，取消註解下方的Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看[此處](https://github.com/doggy8088/generative-ai/tree/main/setup-env)的設定說明。


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

## 程式碼聊天


Vertex AI Codey API 提供一個程式碼聊天 API，專為量身打造適用於編碼情境的多次對話。利用生成式 AI 基礎模型「codechat-bison」與程式碼聊天 API 進行介接，並建立提示，以啟動基於聊天機器人的程式碼對話。本指南將引導你瞭解使用「codechat-bison」模型建立有效提示以參與與程式碼相關的聊天機器人對話的歷程。


### 匯入函式庫


**僅限 Colab：** 取消下方單元格的註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此動作。


In [ ]:
# import vertexai

# PROJECT_ID = ""  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from IPython.display import Markdown, display
from vertexai.language_models import CodeChatModel

## 用 codechat-bison 進行程式碼聊天


「codechat-bison」模型可讓你在多輪對話中從程式碼背景進行自由對話。應用程式會追蹤對話中先前說過的話。因此，如果你期望在應用程式中使用對話進行程式碼生成，請使用「codechat-bison」模型，因為該模型已針對多輪對話使用案例進行微調。


### 載入模型


In [ ]:
code_chat_model = CodeChatModel.from_pretrained("codechat-bison@002")

### 開始聊天會話


In [ ]:
code_chat = code_chat_model.start_chat()

### 傳送訊息
一旦建立好連線，你可以傳送提示，並讓模型按照指示產生產出，並記住脈絡。


In [ ]:
print(
    code_chat.send_message(
        "Please help write a function to calculate the min of two numbers in python",
    ).text
)

你可以看到它知道上一步中產生的程式碼。


In [ ]:
print(
    code_chat.send_message(
        "can you explain the code line by line?",
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        "can you add docstring, typehints and pep8 formating to the code?",
    ).text
)

## 使用案例


### 程式除錯


如果你想降低回應的變異，請保持 temperature=0。如果你想要更多範例 (輸出)，請保持高於 0.2。


In [ ]:
code_chat = code_chat_model.start_chat(temperature=0, max_output_tokens=2048)

print(
    code_chat.send_message(
        '''
        Debug the following scenario based on the problem statement, logic, code and error. Suggest possible cause of error and how to fix that.
        Expalin the error in detail.

        Problem statement: I am trying to write a Python function to implement a simple recommendation system.
        The function should take a list of users and a list of items as input and return a list of recommended items for each user.
        The recommendations should be based on the user's past ratings of items.

        Logic: The function should first create a user-item matrix, where each row represents a user and each column represents an item.
        The value of each cell in the matrix represents the user's rating of the item.
        The function should then use a recommendation algorithm, such as collaborative filtering or content-based filtering, \
        to generate a list of recommended items for each user.

        Code:
        ```
        import numpy as np

        def generate_recommendations(users, items):
          """Generates a list of recommended items for each user.

          Args:
            users: A list of users.
            items: A list of items.

          Returns:
            A list of recommended items for each user.
          """

          # Create a user-item matrix.
          user_item_matrix = np.zeros((len(users), len(items)))
          for user_index, user in enumerate(users):
            for item_index, item in enumerate(items):
              user_item_matrix[user_index, item_index] = user.get_rating(item)

          # Generate recommendations using a recommendation algorithm.
          # ...

          # Return the list of recommended items for each user.
          return recommended_items

        # Example usage:
        users = [User1(), User2(), User3()]
        items = [Item1(), Item2(), Item3()]

        recommended_items = generate_recommendations(users, items)

        print(recommended_items)
        ```
        Error:
        AttributeError: 'User' object has no attribute 'get_rating'

                ```
        ''',
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
       can you re-write the function to address the bug of conversion to int inside the function itself?
        """,
    ).text
)

### 程式碼重構


In [ ]:
code_chat = code_chat_model.start_chat(max_output_tokens=2048)

print(
    code_chat.send_message(
        """
        Given the following C++ code snippet:
        ```c++
        class User {
        public:
          User(const std::string& name, int age)
            : name_(name), age_(age) {}

          std::string GetName() const { return name_; }
          int GetAge() const { return age_; }

        private:
          std::string name_;
          int age_;
        };

        // This function takes a vector of users and returns a new vector containing only users over the age of 18.
        std::vector<User> GetAdultUsers(const std::vector<User>& users) {
          std::vector<User> adult_users;
          for (const User& user : users) {
            if (user.GetAge() >= 18) {
              adult_users.push_back(user);
            }
          }
          return adult_users;
        }
        ```
        Refactor this code to make it more efficient and idiomatic.
        Make sure to identify and fix potential problems.
        Explain the refactoring step by step in detail.
        List down potential changes that can be recommended to the user.
        """,
    ).text
)

### 程式碼檢驗


In [ ]:
code_chat = code_chat_model.start_chat(temperature=0, max_output_tokens=2048)

print(
    code_chat.send_message(
        """
        provide the code review line by line for the following python code: \n\n
```
# Import the requests and json modules
import requestz
import JSON

# Define a class called User
class User:
    # Define a constructor that takes the user's ID, name, and email as arguments
    def __init__(self, id, name, email):
        # Set the user's ID
        self.userId = id

        # Set the user's name
        self.userName = name

        # Set the user's email
        self.userEmail = email

    # Define a method called get_posts that gets the user's posts from the API
    def getPosts(self):
        # Create a URL to the user's posts endpoint
        url = "https://api.example.com/users/{}/posts".format(self.userId)

        # Make a GET request to the URL
        response = requestz.get(url)

        # Check if the response status code is 200 OK
        if response.statusCode != 200:
            # Raise an exception if the response status code is not 200 OK
            raise Exception("Failed to get posts for user {}".format(self.userId))

        # Convert the response content to JSON
        posts = JSON.loads(response.content)

        # Create a list of Posts
        postList = []

        # Iterate over the JSON posts and create a Post object for each post
        for post in posts:
            # Create a new Post object
            newPost = Post(post["id"], post["title"], post["content"])

            # Add the new Post object to the list of Posts
            postList.append(newPost)

        # Return the list of Posts
        return postList

# Define a class called Post
class Post:
    # Define a constructor that takes the post's ID, title, and content as arguments
    def __init__(self, id, title, content):
        # Set the post's ID
        self.postId = id

        # Set the post's title
        self.postTitle = title

        # Set the post's content
        self.postContent = content

# Define a main function
def main():
    # Create a User object for John Doe
    user = User(1, "John Doe", "john.doe@example.com")

    # Get the user's posts
    posts = user.getPosts()

    # Print the title and content of each post
    for post in posts:
        print("Post title: {}".format(post.postTitle))
        print("Post content: {}".format(post.postContent))

# Check if the main function is being called directly
if __name__ == "__main__":
    # Call the main function
    main()
```

        """,
    ).text
)

### 編碼學習


In [ ]:
code_chat = code_chat_model.start_chat(
    temperature=0,
    max_output_tokens=2048,
)

print(
    code_chat.send_message(
        '''
    I am new to Python and i have not read advanced concepts as of now. can you explain this code line by line.
    Include fundamental explanation of some of the advance concepts used in the code as well.
    Also provide an explanation as why somebody made a choice of using complex code vs simple code.  \n\n

    ```
    import functools

    def memoize(func):
      """Memoizes a function, caching its results for future calls.

      Args:
        func: The function to memoize.

      Returns:
        A memoized version of func.
      """

      cache = {}

      @functools.wraps(func)
      def memoized_func(*args, **kwargs):
        key = tuple(args) + tuple(kwargs.items())
        if key in cache:
          return cache[key]
        else:
          result = func(*args, **kwargs)
          cache[key] = result
          return result

      return memoized_func

    def lru_cache(maxsize=128):
      """A least recently used (LRU) cache decorator.

      Args:
        maxsize: The maximum number of items to keep in the cache.

      Returns:
        A decorator that wraps a function and caches its results. The least recently
        used results are evicted when the cache is full.
      """

      def decorating_function(func):
        cache = {}
        queue = collections.deque()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
          key = tuple(args) + tuple(kwargs.items())

          if key in cache:
            value = cache[key]
            queue.remove(key)
            queue.appendleft(key)
            return value

          value = func(*args, **kwargs)
          cache[key] = value
          queue.appendleft(key)

          if len(cache) > maxsize:
            key = queue.pop()
            del cache[key]

          return value

        return wrapper

      return decorating_function
    ```

    ''',
    ).text
)

### 程式碼範本


In [ ]:
code_chat = code_chat_model.start_chat(temperature=0, max_output_tokens=2048)

print(
    code_chat.send_message(
        """
        Write a boilerplate code for FastAPI to serve Llama 7b llm using huggingface locally. Add extra with some boilerplate and #todo for user to fill later:
        - input validation steps,
        - caching user inputs,
        - health check of API,
        - database connection with redis server and
        - Database connection google cloud SQL,  and
        - load balance features

        Also, add some test cases that can check the functionality of the Api endpoint with examples.
        """,
    ).text
)

## 提示設計模式，適用於聊天


### 驗證鏈

驗證鏈 (CoVe) 提示是一種技術，透過使用自我驗證程序來提煉大型語言模型 (LLM) 產生的程式碼。其目標是減輕產生式程式碼中出現幻覺或不準確的可能性。

CoVe 程序包含四個關鍵步驟：

1. 起草初始回應：LLM 根據提供的自然語言描述產生初始程式碼回應。

2. 規劃驗證問題：擬定一系列驗證問題，以仔細檢視初始程式碼回應的準確性和完整性。

3. 執行驗證：驗證問題由 LLM 本身或外部來源獨立回答，以最大程度降低驗證程序中潛在的偏差。

4. 產生最終驗證回應：根據驗證問題的答案，LLM 將精煉初始程式碼回應，以產生最終更準確和可靠的程式碼輸出。

與傳統提示方法相比，CoVe 提示在程式碼產生任務中展現出進步，進而產生更準確和可靠的程式碼輸出。


In [ ]:
code_chat = code_chat_model.start_chat(max_output_tokens=2048)

print(
    code_chat.send_message(
        """
        You are a software developer who can take instructions and follow them to generate and modify code.
        Your goal is to generate code based on what a user has asked, and to keep modifying the code based on the user's verification rules.
        Verification rules are not the same as test functions or test cases.
        Instead, they are steps that the user provides to ensure that the code meets their requirements.

        For example, if a user asks you to generate a code to calculate the factorial of a number:
          Step 1: Initial Setup for function 'calculate_n_factorial'
            - Add input:
              - n: number
            - variables
              - temp: store temporary values
          As, first step, generate a code to calculate the factorial of a number and setup the function and variables.
        and then provides the following verification rules:
          Step 2: Verification steps for the factorial function
            - The code should return 1 for the input 0.
            - The code should return 2 for the input 1.
            - The code should return 6 for the input 3.
        Now you would modify the code to ensure that it meets the verification rules.

        It’s very important to adjust each and every verification in the modification of the code. Each time when the code is modified,
        explain your processes. Your job is to self-reflect and correct based on the user input and verification rule.
        Do not add anything from your end, just follow user input.
        Respond to this context with “Yes, I understand” and do not add any code at this stage. Wait for next instructions.

        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 1:
            - Python function ‘calculate_total_cost_parcel’
            - Add Input:
                - weight
                - distance
                - shipping_method
                - insurance_coverage
                - discount_code
            - Add Variable:
            base_shipping_cost: weight * distance * shipping_method
            shipping_method_multiplier = { "standard": 1.0, "expedited": 1.5, "overnight": 2.0 }
            insurance_coverage_multiplier =  { "none": 1.0, "basic": 1.1, "premium": 1.2 }
            shipping_cost = shipping_method_multiplier[shipping_method]
            insurance_cost = insurance_coverage_multiplier[insurance_coverage]
            shipping_cost = base_shipping_cost*shipping_cost *insurance_cost
            discount = 0
            total_cost = shipping_cost-discount [this is what function will return]

        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 2: Verification steps for the type hints, docstring, and description for the input to the function:
        weight: The weight of the package in kilograms.
        distance: The distance the package will be shipped in kilometres.
        shipping_method: The shipping method, which can be one of the following:
              - “standard": Standard shipping, which takes 3-5 business days.
              - “expedited": Expedited shipping, which takes 1-2 business days.
              -  "overnight": Overnight shipping, which takes 1 business day.
        insurance_coverage: The insurance coverage, which can be one of the following:
              - “none": No insurance coverage.
              -“ basic": Basic insurance coverage, which covers up to $100 in losses.
              - "premium": Premium insurance coverage, which covers up to $500 in losses.
        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 3: Verification steps for the input to the function:
          - Check if the weight is non-negative.
          - Check if the distance is non-negative.
          - Check if the shipping method is valid.
          - Check if the insurance coverage is valid.
          - Check if the discount code is valid.
        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 4: Verification for Discount
          - If the discount code is "SHIP10", multiply the base shipping cost by 0.10 and subtract the result from the total shipping cost.
          - If the discount code is "SHIP20", multiply the base shipping cost by 0.20 and subtract the result from the total shipping cost.
          - Otherwise, the discount code is invalid, so do not apply any discount.
        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 5: Generate test cases that can be used to test the function ‘calculate_total_cost_parcel’. The test cases \
        should include incorrect inputs, unexpected inputs, edge cases that are generally not thought by a developer or a QA.

        """,
    ).text
)

In [ ]:
print(
    code_chat.send_message(
        """
        How did all the verification steps improve the '‘calculate_total_cost_parcel’' function that was generated?
        Explain in details with example, code before and after, and bullet points.
        """,
    ).text
)

### 自洽

自洽提示是一種技術，它利用大語言模型 (LLM) 識別和支援推理中一致模式的能力，來增強大語言模型所產生的程式碼品質。它旨在透過引入一種機制來選擇多個可能選項中最一致且最可靠的程式碼輸出，從而解決不一致或錯誤的程式碼生成問題。

自洽提示過程包含三個關鍵步驟：

* 生成多條推理路徑：LLM 會生成多條不同的推理路徑，它們代表瞭解決給定程式碼生成任務的不同方法。

* 評估一致性：對於每條推理路徑，LLM 都會評估其中間步驟和最終程式碼輸出的的連貫性。這包括識別模式、檢查矛盾，並確保與自然語言描述保持一致。

* 選擇最一致的回應：基於一致性評估，LLM 會選擇展現最高級別的一致性並產生最可靠程式碼輸出的推理路徑。

自洽提示已顯示出在改善所生成程式碼的準確性和可靠性方面有效，尤其是在複雜或含糊不清的任務中。它已被證明可以減少不一致性和錯誤的發生，從而導致更健壯和值得信賴的程式碼生成。


In [ ]:
code_chat = code_chat_model.start_chat(max_output_tokens=2048)

print(
    code_chat.send_message(
        """
        Input: any english words or group of characters.
        Output: reverse of the input string.

        Goal:
          1) Generate 3 different python code snippets for reverse_string() based on algorithmic complexity and mentioning it along the code.
          2) For each code snippet add typehints, docstrings, classes if required, pep8 formatting.
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Going forward, i want you to follow each instruction one by one based on the code that is generated in the previous steps:

        Step 1:  For each code snippet, generate a test case that checks if the function reverses the string correctly. The test cases \
        should include incorrect inputs, unexpected inputs, edge cases that are generally not thought by a developer or a QA.
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 2: For each code snippet, Intergate the exception handling for incorrect inputs, unexpected inputs, \
        edge cases that based on previous step and re-write the functions
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 3 : Based on the test written, code completeness and algorithm complexity, select the code which is best

        Step 4:  Explain the reasoning in detail as bullet points of why this is selected compared to other options.
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Step 5: Show the code which is selected along with its test cases.
        """,
    )
)

### 思考樹

思考樹 (ToT) 提示法是一種技術，用於指導大型語言模型 (LLM) 透過將任務分解成中間自然語言步驟的分層結構來生成程式碼。這種方法旨在解決傳統提示法侷限性，可能會導致 LLM 陷入局部最佳化或產生結構不佳或未最佳化的程式碼。

ToT 提示流程涉及三個關鍵步驟：

* 分解任務：以自然語言描述任務，再進一步分解成一系列較小的子任務，形成樹狀結構。

* 產生中間想法：對於樹中的每個子任務，LLM 產生相應的中間想法，這是說明如何解決該子任務的自然語言說明。

* 建構程式碼：LLM 將中間想法組合成一個有凝聚力且有條理的程式碼輸出，遵循樹的分層組織。

在程式碼生成任務中，ToT 提示法已證明優於傳統提示法，特別是針對複雜或多步驟的問題。它幫助 LLM 以更結構化和系統化的方式對問題進行推理，進而產生更有效且可靠的程式碼。


In [ ]:
code_chat = code_chat_model.start_chat(max_output_tokens=2048, temperature=0.5)

print(
    code_chat.send_message(
        """
        Imagine a tree of thoughts, where each thought represents a different step in the data preprocessing pipeline.
        The goal of this pipeline is to run a regression model on a ecommerce data from bigquery.
        Start at the root of the tree, and write down a thought that captures the main goal of the data preprocessing pipeline.
        Then, branch out from that thought and write down two more thoughts that represent related steps in the pipeline.
        Continue this process until you have a complete tree of thoughts, with each leaf representing a single line of Python code.
        For each branch and leaf, only write the thoughts and not code. Do not write code for each branch and leaves and put them in proper markdown.
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        The data also needs to be joined across different tables in BigQuery before starting pre-processing.
        For example customer table has to be merged with the order table.
        this should be added at the initial branchess a thought.
        After that Add more branches for model building using BQML once the data is scaled.
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Reconfigure the branches from the root as per the newly added thoughts. Follow the proper flow. rewrite the whole branches and leaves
        """,
    )
)

In [ ]:
print(
    code_chat.send_message(
        """
        Generate the code for each branch and leaves.
        """,
    )
)